# Loaders

In [227]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=["https://www.ibm.com/topics/large-language-models",
                                    "https://www.cloudflare.com/en-in/learning/ai/what-is-large-language-model/",
                                    "https://www.elastic.co/what-is/large-language-models",
                                    "https://en.wikipedia.org/wiki/Large_language_model"
                                    ])
data = loader.load()

In [228]:
len(data)

4

# Text splitters

In [229]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " "],
                                            chunk_size=500,
                                            chunk_overlap=200
)

chunks = text_splitter.split_documents(data)

In [230]:
len(chunks)

392

# Vector Database

In [231]:
from sentence_transformers import SentenceTransformer
import pickle
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
# Load a pre-trained Sentence-BERT model
model = SentenceTransformer('all-roberta-large-v1')
embeddings = model.encode([chunk.page_content for chunk in chunks])

# Create FAISS vector store from documents and embeddings

text_embedding_pairs = zip([chunk.page_content for chunk in chunks], embeddings)
vectorstore = FAISS.from_embeddings(text_embedding_pairs, model)
    
file_path = "vector_index.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(vectorstore, f)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\ravip\anaconda3\envs\SentimentAnalysis\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ravip\.cache\huggingface\hub\models--sentence-transformers--all-roberta-large-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
from dotenv import load_dotenv
import langchain
import os

# Load environment variables from .env file
load_dotenv()

In [ ]:
# Access the Hugging Face API key
huggingface_api_key = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
llm = HuggingFaceHub(huggingfacehub_api_token=huggingface_api_key,
                     repo_id="google/flan-t5-large",
                     model_kwargs={"temperature": 0.8, "max_length": 500},
                     task="text2text-generation"
                    )

In [ ]:
def get_context(query_text, embedding_model, vectorstore, chunks):
    # Encode the query text
    query_text = "What is LLM, what are its uses?"
    query_vector = embedding_model.encode([query_text])

    # Perform the search
    k = 5  # Number of nearest neighbors to retrieve
    distances, indices = vectorstore.index.search(query_vector, k)

    docs = []
    # Retrieve and print the results
    for i in range(k):
        doc_index = indices[0][i]
        distance = distances[0][i]
#         print(f"Result {i+1}:")
#         print(f"Text: {chunks[doc_index].page_content}")
#         print(f"Distance: {distance}")
        docs.append(chunks[doc_index].page_content)
    return ". ".join(docs)

In [ ]:
# Define a simple prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI assistant tasked with answering questions based on the given context. 
    Provide a concise and accurate answer to the question asked.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
    )

# Create an LLM chain with the prompt template and the LLM
llm_chain = LLMChain(prompt=prompt_template, llm=llm)

query_text = "What is LLM, what are its uses?"
context = get_context(query_text, model, vectorstore, chunks)
response = llm_chain.invoke({"context": context, "question": query_text})

# Print the response
print("Answer:", response['text'])

In [ ]:
query_text = "How LLM Model performance can be increased"
context = get_context(query_text, model, vectorstore, chunks)
response = llm_chain.invoke({"context": context, "question": query_text})

# Print the response
print("Answer:", response['text'])

In [ ]:
response

In [ ]:
query_text = "when was transformer architecture invented?"
context = get_context(query_text, model, vectorstore, chunks)
response = llm_chain.invoke({"context": context, "question": query_text})

# Print the response
print("Answer:", response)